In [ ]:
import numpy as np
import pyemma
import projX
import simplex
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
trajfile = 'data/c-alpha_centered.stride.100.xtc'
top = 'data/bpti-c-alpha_centered.pdb'

In [ ]:
feat = pyemma.coordinates.featurizer(top)
pairs = feat.pairs(range(feat.topology.n_atoms)[::2])
feat.add_distances(pairs)
src  = pyemma.coordinates.source(trajfile, features=feat)
tica = pyemma.coordinates.tica(src, lag=100, dim=5)
Y = tica.get_output()[0]

In [ ]:
vertices, means = simplex.find_vertices_inner_simplex(Y, return_means=True)
P, o = simplex.splash_corner_projection(vertices, center=0)
np.save('data/Z.npy', Z)

In [ ]:
ax, fig, iwd, sample, geom = projX.visualize_FES(trajfile, top, 'data/Z.npy', 
                                          )
iwd

In [ ]:
pyemma.plots.plot_free_energy(Z[:, 0], Z[:, 1])
V = (vertices-o).dot(P)
plt.scatter(V[:, 0], V[:, 1], label='vertices')
M = (means-o).dot(P)
plt.scatter(M[:, 0], M[:, 1], c='r', label='core means')
plt.legend()

In [ ]:
dtrajs = simplex.core_assignments(Y, vertices, f=0.5)

In [ ]:
C, s = simplex.milestoning_count_matrix(dtrajs, lag=5, return_scrapped=True)
s, C

In [ ]:
import msmtools
msmtools.estimation.largest_connected_set(C)